In [2]:
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.chart import PieChart, Reference
from collections import defaultdict

def copy_data_and_assign_headings_and_summarize_land_use():
    # Load the workbook and worksheets
    workbook = openpyxl.load_workbook('LandUse_Summary.xlsx') # name of the file depends on the user's preference
    ws_source1 = workbook['hru.con'] # the data copied from hru.con file of SWAT+
    ws_source2 = workbook['landuse.lum'] # the data copied from landuse.lum file of SWAT+
    
    # Create a new worksheet for HRU_CN
    ws_destination = workbook.create_sheet(title="HRU_CN")
    
    # Copy data from the first source column and assign a new heading
    ws_destination['A1'] = "Name"
    ws_destination['A1'].font = Font(bold=True)
    ws_destination['A1'].fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    for i, row in enumerate(ws_source1.iter_rows(min_row=3, min_col=3, max_col=3, values_only=True), start=2):
        ws_destination.cell(row=i, column=1, value=row[0])
    
    # Copy data from the second source column and assign a new heading
    ws_destination['B1'] = "Area"
    ws_destination['B1'].font = Font(bold=True)
    ws_destination['B1'].fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    for i, row in enumerate(ws_source1.iter_rows(min_row=3, min_col=5, max_col=5, values_only=True), start=2):
        ws_destination.cell(row=i, column=2, value=row[0])
    
    # Copy data from the third source column and assign a new heading
    ws_destination['C1'] = "Land Use"
    ws_destination['C1'].font = Font(bold=True)
    ws_destination['C1'].fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    for i, row in enumerate(ws_source2.iter_rows(min_row=3, min_col=6, max_col=6, values_only=True), start=2):
        ws_destination.cell(row=i, column=3, value=row[0])
    
    # Center align all cells in the destination sheet
    for row in ws_destination.iter_rows():
        for cell in row:
            cell.alignment = Alignment(horizontal='center')
    
    # Create a new worksheet for LU_Summary
    ws_summary = workbook.create_sheet(title="LU_Summary")
    
    # Initialize summary table
    headers = [("A1", "Land Use"), ("B1", "Total Area"), ("C1", "Percentage")]
    for cell, value in headers:
        ws_summary[cell] = value
        ws_summary[cell].font = Font(bold=True)
        ws_summary[cell].fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
        ws_summary[cell].alignment = Alignment(horizontal='center')
    
    # Get unique land uses and calculate total area
    land_use_col = 3
    area_col = 2
    data_rows = list(ws_destination.iter_rows(min_row=2, min_col=1, max_col=3, values_only=True))
    total_area = sum(row[1] for row in data_rows if row[1] is not None)
    land_use_areas = defaultdict(float)
    
    for row in data_rows:
        land_use, area = row[2], row[1]
        if land_use and area:
            land_use_areas[land_use] += area
    
    # Fill summary table
    for i, (land_use, area) in enumerate(land_use_areas.items(), start=2):
        ws_summary.cell(row=i, column=1, value=land_use)
        ws_summary.cell(row=i, column=2, value=area)
        ws_summary.cell(row=i, column=3, value=(area / total_area) * 100)
    
    # Autofit columns
    for col in ws_summary.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws_summary.column_dimensions[column].width = adjusted_width
    
     # Create pie chart
    pie = PieChart()
    labels = Reference(ws_summary, min_col=1, min_row=2, max_row=len(land_use_areas) + 1)
    data = Reference(ws_summary, min_col=3, min_row=1, max_row=len(land_use_areas) + 1)
    pie.add_data(data, titles_from_data=True)
    pie.set_categories(labels)
    pie.title = "Land Use Percentage"

    # Add the chart to the sheet
    ws_summary.add_chart(pie, "E5")
    
    # Save the workbook
    workbook.save('SWAT+LandUse_Summary.xlsx')
    print("Summary generated successfully!")
    
copy_data_and_assign_headings_and_summarize_land_use()

Summary generated successfully!
